## From [ipywidget documentation]((https://ipywidgets.readthedocs.io/en/stable/embedding.html)
### 1 - Minimal export
+ Run Cell
+ Open generated file in current directory: **export-a.html**

In [1]:
from ipywidgets import IntSlider
from ipywidgets.embed import embed_minimal_html

slider = IntSlider(value=40)
embed_minimal_html('export-a.html', views=[slider], title='Widgets export')

### 2 - General export
+ Run Cell
+ Open generated file in current directory: **export-b.html**

In [2]:
import json

from ipywidgets import IntSlider
from ipywidgets.embed import embed_data

from widget_d3_slider import Slider as D3Slider

s1 = IntSlider(max=200, value=100)
s2 = IntSlider(value=40)
d3 = D3Slider(value=33)
data = embed_data(views=[s1, d3, s2])

html_template = """
<html>
  <head>

    <title>Widget export</title>

    <!-- Load RequireJS, used by the IPywidgets for dependency management -->
    <script 
      src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" 
      integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" 
      crossorigin="anonymous">
    </script>

    <!-- Load IPywidgets bundle for embedding. -->
    <script 
      src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" 
      crossorigin="anonymous">
    </script>


    <!-- The state of all the widget models on the page -->
    <script type="application/vnd.jupyter.widget-state+json">
      {manager_state}
    </script>
  </head>

  <body>

    <h1>Widget export</h1>

    <div id="first-slider-widget">
      <!-- This script tag will be replaced by the view's DOM tree -->
      <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[0]}
      </script>
    </div>

    <hrule />

    <div id="d3-slider-widget">
      <!-- This script tag will be replaced by the view's DOM tree -->
      <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[1]}
      </script>
    </div>

    <hrule />

    <div id="second-slider-widget">
      <!-- This script tag will be replaced by the view's DOM tree -->
      <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[2]}
      </script>
    </div>

  </body>
</html>
"""

manager_state = json.dumps(data['manager_state'])
widget_views = [json.dumps(view) for view in data['view_specs']]
rendered_template = html_template.format(manager_state=manager_state,
                                         widget_views=widget_views)
with open('export-b.html', 'w') as fp:
    fp.write(rendered_template)
    

### 3 - HTTP Server
+ Choose a file to rename from those created above: **export-a.html**, **export-b.html**
+ Run Cell
+ Run generated script to launch http server: **server.sh**

In [3]:
import os, sys, shutil

if sys.platform in ['darwin', 'linux']:
    suffix = 'sh'
elif sys.platform == 'win32':
    suffix = 'bat'
else:
    print('OS unknown: No web server launch script created')

# rename file
file_path = 'export-b.html'
with open(file_path, 'r') as f:
    content = f.read()
with open('index.html', 'w') as f:
    f.write(content)

# copy d3-slider dist/index.js to current directory
shutil.copy(os.path.join('..', 'jupyter-widget-d3-slider', 'js', 'dist', 'index.js'), 
            os.path.join('.', 'd3-slider-index.js'))

content = 'python -m http.server 8089'
file_path = os.path.join('.', 'server.' + suffix)
with open(file_path, 'w') as f:
    f.write(content)